<a href="https://colab.research.google.com/github/rhailper/milestoneII/blob/main/notebooks/SIADS696_DataExplorationAndCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Connect to drive and pull from github

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
from config import *

In [4]:
#!git clone https://{token}@github.com/rhailper/milestoneII.git

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/milestoneII

/content/drive/MyDrive/Colab Notebooks/milestoneII


In [6]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 1.84 KiB | 5.00 KiB/s, done.
From https://github.com/rhailper/milestoneII
   59eaef5..1a5945d  main       -> origin/main
Updating 59eaef5..1a5945d
Fast-forward
 .../SIADS696_DataExplorationAndCleaning.ipynb      | 1188 +++-----------------
 1 file changed, 147 insertions(+), 1041 deletions(-)


# Import and do basic cleaning to files

In [7]:
import pandas as pd
import numpy as numpy
import itertools
from ast import literal_eval
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

## Client information

In [8]:
# import client info - this file has basic demographic information about the client
demo = pd.read_csv('data/input_data/CLIENT_INFORMATION.csv') 

In [9]:
# drop row with na values 
# If a client has na values in this value that means they died or stopped 
# receiving services from the organization 
# (it might also be interesting to look at these client)
#client_info = client_info.dropna()

Due to the deidentification process, this dataset was not allowed to disclose the exact age of any client over the age of 90. Any client over the age of 90 was coded as 90+ so this needs to be changed in order to make the feature numerical.

In [10]:
# replace '90+' with 90
demo['Age'] = demo['Age'].str.replace('90+','90',regex=False)#.astype(int)

In [11]:
# convert categorical variables into one hot encoded dummy variables
df = pd.get_dummies(demo, columns=['Gender','Federal Poverty','Race','Primary Funding Source','Multiple Funding Sources?'])

#### Client services


In [12]:
# import client services - this files contains service utilization
serv = pd.read_csv('data/input_data/CLIENT_SERVICES.csv') 

In [13]:
# convert service column from string to list
serv['Services'] = serv['Services'].apply(literal_eval)

In [14]:
# get only 2022
serv = serv[serv['Year']==2022]

In [15]:
# get the total service cost per client 2022
service_costs_total = pd.DataFrame(serv.groupby('ID')['Cost of Serivces'].sum()).reset_index()
service_costs_total.columns = ['ID','service_costs_total']

In [16]:
# get the avg service cost per month per client 2022
service_costs_avg = pd.DataFrame(serv.groupby('ID')['Cost of Serivces'].mean()).reset_index()
service_costs_avg.columns=['ID','service_costs_avg']

In addition to the costs of services for each client, from this table we can get the number and type of services each client recieves. The types of services will need to be one hot encoded. 

In [17]:
# get list of all unique services a client had in 2022
# get a list of unique services by client
num_unique_services = pd.DataFrame(serv.groupby('ID').apply(lambda x : len(set(x['Services'].sum())))).reset_index().rename({0:'num_unique_services'},axis=1)

In [18]:
# get the list of services as one hot encodings
unique_services = serv.groupby('ID').apply(lambda x : set(x['Services'].sum()))
mlb = MultiLabelBinarizer()

unique_services_one_hot = pd.DataFrame(mlb.fit_transform(unique_services),
                   columns=mlb.classes_,
                   index=unique_services.index)
unique_services_one_hot = unique_services_one_hot.reset_index()

In [19]:
# add new features to main df
df = df.merge(service_costs_total,how='left')
df = df.merge(service_costs_avg,how='left')
df = df.merge(num_unique_services,how='left')
df = df.merge(unique_services_one_hot,how='left')

#### Diagnoses

In [20]:
# import diagnoses - this gile contains client diagnoses based on ICD-10 codes
diag = pd.read_csv('data/input_data/DIAGNOSES.csv') 

#### Questionaire

In [21]:
# import questionaire - this file contains information about clients ability to complete daily activities 
quest = pd.read_csv('data/input_data/QUESTIONAIRE.csv') 

In [22]:
# get the most recent annual questionaire for each client 
# keep only the long form of the questionaire (removes nas)
quest = quest[quest['AssesmentType']!=2] 

# convert InterRAI Period to ordered categorical to be able to sort by date
cat_type = CategoricalDtype(categories=[ 'JUL_DEC2020','JAN_JUN2021', 
                                        'JUL_DEC2021','JAN_JUN2022', 
                                        'JUL_DEC2022'],ordered=True)
quest['InterRAI Period'] = quest['InterRAI Period'].astype(cat_type)

# sort newest to oldest
quest = quest.sort_values('InterRAI Period',ascending=False) 

# get most recent for each client
quest = quest.groupby('ID').first()

In [23]:
# drop irrelevant columns 
quest = quest.drop(['InterRAI Period','AssesmentType','A8_AssessmentReasonType_Value',
            'A10_GoalsOfCare','A10_PrimaryGoal'],axis=1)

In [24]:
# convert categorical to one hot encodings 
quest_one_hot = pd.get_dummies(quest, columns=['A4_MaritalStatusType_Value',
                                               'A12_ResidentialStatusType_Value',
                                               'A13A_LivingArrangementType_Value',
                                              'A13B_LivesSomeoneNewBooleanType_Value',
                                               'B3_PrimaryLanguageType_Value'])

In [25]:
# add to main df
df = df.merge(quest_one_hot,on=['ID'],how='left')

#### Hospitalzations (will be outcome variable for supervised learning)

In [26]:
# import hospitalzations - this file contains information about client hospitalzations in the past 2 years
hosp = pd.read_csv('data/input_data/HOSPITALIZATIONS.csv') 

In [27]:
# get the total number of emergecy room visits in 2022 per client (feature)
emerg_total_2022 = hosp[(hosp['Year']==2022)&(hosp['Admittype']=='Emergency')].groupby('ID')['Number Hospitalzations'].sum()
emerg_total_2022.name = 'emerg_total_2022'

# get the total number of health care visits in 2022 per client (feature)
health_visits_total_2022 = hosp[(hosp['Year']==2022)].groupby('ID')['Number Hospitalzations'].sum()
health_visits_total_2022.name = 'health_visits_total_2022'

# get the average emergency room visits per month in 2022 per client (feature)
emerg_avg_2022 = hosp[(hosp['Year']==2022)&(hosp['Admittype']=='Emergency')].groupby('ID')['Number Hospitalzations'].mean()
emerg_avg_2022.name = 'emerg_avg_2022'

# get the average health care visits per month in 2022 per client (feature)
health_visits_avg_2022 = hosp[(hosp['Year']==2022)].groupby('ID')['Number Hospitalzations'].mean()
health_visits_avg_2022.name = 'health_visits_avg_2022'

# get total number of emergecy room visits per client in Jan,Feb,Mar 2023 (outcome)
emerg_total_2023 = hosp[(hosp['Year']==2023)&(hosp['Admittype']=='Emergency')].groupby('ID')['Number Hospitalzations'].sum()
emerg_total_2023.name = 'emerg_total_2023'

# get average number of emergecy room visits per client in Jan,Feb,Mar 2023 (outcome)
emerg_avg_2023 = hosp[(hosp['Year']==2023)&(hosp['Admittype']=='Emergency')].groupby('ID')['Number Hospitalzations'].mean()
emerg_avg_2023.name = 'emerg_avg_2023'

# get total number of health care visits per client in Jan,Feb,Mar 2023 (outcome)
health_visits_total_2023 = hosp[(hosp['Year']==2023)].groupby('ID')['Number Hospitalzations'].sum()
health_visits_total_2023.name = 'health_visits_total_2023'

# get average number of health care visits per client in Jan,Feb,Mar 2023 (outcome)
health_visits_avg_2023 = hosp[(hosp['Year']==2023)].groupby('ID')['Number Hospitalzations'].mean()
health_visits_avg_2023.name = 'health_visits_avg_2023'

In [28]:
hosp_features = pd.concat([emerg_total_2022,health_visits_total_2022,
                           emerg_avg_2022,health_visits_avg_2022],axis=1).reset_index()
hosp_outcomes = pd.concat([emerg_total_2023, emerg_avg_2023,
                           health_visits_total_2023, health_visits_avg_2023],axis=1).reset_index()

In [29]:
# add features to main df
df = df.merge(hosp_features,on=['ID'],how='left')
# add outcomes to main df
df = df.merge(hosp_outcomes,on=['ID'],how='left')

In [30]:
# fill hosp nas with zeros becuase in this case if it's na then this means the 
# client did not visit the hospital in given time frame
for col in ['emerg_total_2022','health_visits_total_2022',
            'emerg_avg_2022','health_visits_avg_2022',
            'emerg_total_2023', 'emerg_avg_2023',
            'health_visits_total_2023', 'health_visits_avg_2023']:
  df[col] = df[col].fillna(0)

In [31]:
df

,ID,Age,Federal % of Poverty,ADL Count,Critical Need Count,IADL Count,Skilled Need Count,Nutrition Score,Gender_Female,Gender_Male,...,B3_PrimaryLanguageType_Value_3.0,B3_PrimaryLanguageType_Value_4.0,emerg_total_2022,health_visits_total_2022,emerg_avg_2022,health_visits_avg_2022,emerg_total_2023,emerg_avg_2023,health_visits_total_2023,health_visits_avg_2023
0,10,38,87.0,4.0,0.0,5.0,0.0,0.0,0,1,...,0.0,0.0,0.0,3.0,0.000000,1.000000,0.0,0.0,4.0,1.333333
1,100035,52,73.0,6.0,1.0,8.0,0.0,3.0,1,0,...,0.0,0.0,0.0,33.0,0.000000,3.000000,0.0,0.0,11.0,3.666667
2,100048,90,124.0,NaN,NaN,NaN,NaN,NaN,0,1,...,NaN,NaN,2.0,4.0,1.000000,1.000000,0.0,0.0,0.0,0.000000
3,100061,53,51.0,6.0,1.0,6.0,0.0,3.0,1,0,...,0.0,0.0,0.0,9.0,0.000000,1.285714,0.0,0.0,4.0,2.000000
4,100073,69,65.0,0.0,2.0,1.0,0.0,8.0,0,1,...,NaN,NaN,0.0,7.0,0.000000,1.750000,0.0,0.0,2.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13598,99910,67,145.0,5.0,1.0,7.0,0.0,3.0,1,0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
13599,99919,90,96.0,7.0,1.0,8.0,0.0,6.0,1,0,...,0.0,0.0,11.0,35.0,1.833333,2.500000,0.0,0.0,12.0,12.000000
13600,99963,55,76.0,5.0,2.0,6.0,0.0,9.0,1,0,...,0.0,0.0,3.0,25.0,1.000000,2.272727,2.0,1.0,7.0,1.166667
13601,99967,60,72.0,5.0,1.0,5.0,0.0,8.0,1,0,...,0.0,0.0,0.0,22.0,0.000000,2.444444,1.0,1.0,23.0,5.750000


## Saving to github

In [32]:
#%cd /content/drive/MyDrive/Colab Notebooks/milestoneII

In [33]:
#!git config --global user.email "rhailper@umich.edu"
#!git config --global user.name "rhailper"
#!git pull

In [34]:
#!git remote add origin https://{token}@github.com/rhailper/milestoneII.git

In [35]:
#!git pull

In [36]:
#!git checkout main

In [37]:
#!git add .
#!git commit -m 'Updates to folder structure'
#!git push https://{token}@github.com/rhailper/milestoneII.git